In [25]:
import os
import numpy as np

def get_subdirectories(parent_directory):
    """주어진 디렉토리 내의 모든 하위 디렉토리의 경로를 리스트로 반환합니다."""
    subdirectories = [os.path.join(parent_directory, d) for d in os.listdir(parent_directory) 
                      if os.path.isdir(os.path.join(parent_directory, d))]
    return subdirectories

def get_all_subdirectories(parent_directory):
    """주어진 디렉토리 내의 첫 번째 및 두 번째 레벨 하위 디렉토리의 경로를 NumPy ndarray로 반환합니다."""
    first_level_subdirs = get_subdirectories(parent_directory)
    all_subdirs = []

    for subdir in first_level_subdirs:
        second_level_subdirs = get_subdirectories(subdir)
        all_subdirs.extend(second_level_subdirs)  # 두 번째 레벨의 디렉토리를 추가

    return np.array(all_subdirs)



category_list = get_subdirectories(os.path.join(os.getcwd(),"Dataset/Train/Image"))
all_list = get_all_subdirectories(os.path.join(os.getcwd(),"Dataset/Train/Image"))
print(all_list)

['/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Organization/066595'
 '/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Organization/107272'
 '/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Organization/116132'
 ...
 '/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Cutting/086556'
 '/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Cutting/113609'
 '/Users/seungwoo/Workspace/Causal_Inference/Dataset/Train/Image/Cutting/079350']


In [29]:
from Dataset import *
dir = os.path.join(os.getcwd(),"Dataset/Train/Image")
train_dataset = BaseDataset(dir,True)
print(len(train_dataset))
val_dataset = BaseDataset(dir,False)
print(len(val_dataset))

48000
6000
